## Human-in-the-Loop
There are two main ways to interact with the team from your application:

1. During a team’s run – execution of run() or run_stream(), provide feedback through a UserProxyAgent.

2. Once the run terminates, provide feedback through input to the next call to run() or run_stream().

## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

## Create Azure OpenAI Client
Using the model client class

In [2]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Create the token provider
#token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=azure_openai_key, # For key-based authentication.
)

## UserProxyAgent - Providing Feedback During a Run
The UserProxyAgent is a special built-in agent that acts as a proxy for a user to provide feedback to the team.

To use the UserProxyAgent, you can create an instance of it and include it in the team before running the team. The team will decide when to call the UserProxyAgent to ask for feedback from the user.

When UserProxyAgent is called during a run, it blocks the execution of the team until the user provides feedback or errors out. This will hold up the team’s progress and put the team in an unstable state that cannot be saved or resumed.

Due to the blocking nature of this approach, it is recommended to use it only for short interactions that require immediate feedback from the user, such as asking for approval or disapproval with a button click, or an alert requiring immediate attention otherwise failing the task.

In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback.",
)

# Create the user proxy agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([primary_agent, critic_agent, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
In autumn’s golden haze, the world transforms,
Leaves drift like whispers through the cooling air,
A tapestry of amber, garnet forms,
And nature paints her scene with tender care.

The harvest moon spills light upon the trees,
While pumpkins line the fields in orange repose,
A scent of spice is carried by the breeze,
And woodsmoke curls where fading daylight goes.

The world slows down beneath a twilight sky,
Where constellations gleam with frosty breath,
And quiet nights invite the heart’s deep sigh,
Embracing fall’s sweet song of life and death.
[Prompt tokens: 28, Completion tokens: 117]


/home/codespace/.python/current/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:343: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-05-13. Model mapping may be incorrect.
  result = await self._model_client.create(


---------- critic ----------
Your poem beautifully captures the essence and imagery of the fall season. The vivid descriptions and delicate language paint a picture that is both serene and evocative. Here are a few points of feedback to consider:

1. **Imagery and Language**: Your use of descriptive language is excellent. Phrases like "autumn’s golden haze" and "tapestry of amber, garnet" are visually and emotionally evocative, bringing the reader directly into the fall scene.

2. **Structure**: The poem’s structure is consistent and flows well. The traditional quatrains give it a rhythmic, almost nostalgic feel that suits the theme of fall. 

3. **Mood**: You've effectively created a contemplative and serene mood, which suits the season of fall as a time of transition and change.

4. **Specificity**: Consider including more specific elements or imagery unique to autumn that might resonate universally. This could enhance the personal connection for readers, such as calling out specific

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=117), content='In autumn’s golden haze, the world transforms,\nLeaves drift like whispers through the cooling air,\nA tapestry of amber, garnet forms,\nAnd nature paints her scene with tender care.\n\nThe harvest moon spills light upon the trees,\nWhile pumpkins line the fields in orange repose,\nA scent of spice is carried by the breeze,\nAnd woodsmoke curls where fading daylight goes.\n\nThe world slows down beneath a twilight sky,\nWhere constellations gleam with frosty breath,\nAnd quiet nights invite the heart’s deep sigh,\nEmbracing fall’s sweet song of life and death.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=148, completion_tokens=335), content='Your poem beautifully captures the essence and imagery 

## Using Max Turns

This method allows you to pause the team for user input by setting a maximum number of turns. 

To implement this, set the max_turns parameter in the RoundRobinGroupChat() constructor.

Once the team stops, the turn count will be reset. 

When you resume the team, it will start from 0 again. However, the team’s internal state will be preserved, for example, the RoundRobinGroupChat will resume from the next agent in the list with the same conversation history.

In [4]:
# Create the team setting a maximum number of turns to 2.
# One chat by agent is one turn. So 2 is needed for the two agents
team = RoundRobinGroupChat([primary_agent, critic_agent], max_turns=2)

task = "Write a short poem about the fall season."
while True:
    # Run the conversation and stream to the console.
    stream = team.run_stream(task=task)
    # Use asyncio.run(...) when running in a script.
    await Console(stream)
    # Get the user response.
    task = input("Enter your feedback (type 'exit' to leave): ")
    if task.lower().strip() == "exit":
        break

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
In autumn’s wistful glow, the leaves descend,
Like whispered secrets from the trees they fall,
A symphony of colors blend and bend,
As nature heeds her ancient, mystic call.

The crisp air carries scents of earth and wood,
Pumpkins and harvest fields in golden light,
Each fleeting day embraced, well understood,
As sunset frames the early autumn night.

Under the starlit sky, the world takes pause,
The quiet whispers of the season grown,
A time for warmth, reflection, and because
In fall, the heart finds solace in its own.
[Prompt tokens: 969, Completion tokens: 116]
---------- critic ----------
Your poem captures the essence of fall beautifully, with rich imagery and a contemplative tone. Here are some points of feedback to consider:

1. **Imagery and Language**: Your use of descriptive language is vivid and evocative. Phrases like "wisftul glow" and "symphony of colors" bring the imagery

## HandoffTermination 
HandoffTermination stops the team when an agent sends a HandoffMessage message.

Note: The model used with AssistantAgent must support tool call to use the handoff feature.

In [5]:
from autogen_agentchat.base import Handoff
from autogen_agentchat.conditions import HandoffTermination

# Create the agents.
# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=az_model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=az_model_client,
    system_message="Provide constructive feedback first then transfer to user.",
    handoffs=[Handoff(target="user", message="Transfer to user.")],
)

# Define a termination condition that checks for handoff message targetting helper OR text "APPROVE".
handoff_termination = HandoffTermination(target="user")

# Create the team.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=handoff_termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a short poem about the fall season.")
# Use asyncio.run(...) when running in a script.
await Console(stream)


---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Golden leaves in dance so fair,
Crisp whispers brush the cooling air.
Pumpkin hues and sunset skies,
Autumn's beauty fills our eyes.

Harvest moons and firelight glow,
Warmth amidst the evening's cold.
Nature's breath, a gentle sigh,
Fall's embrace bids summer goodbye.
[Prompt tokens: 28, Completion tokens: 61]
---------- critic ----------
[FunctionCall(id='call_biBQto5IJvmwh1AR9utw0Dtc', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 123, Completion tokens: 88]
---------- critic ----------
[FunctionExecutionResult(content='Transfer to user.', call_id='call_biBQto5IJvmwh1AR9utw0Dtc')]
---------- critic ----------
Transfer to user.
---------- Summary ----------
Number of messages: 5
Finish reason: Handoff to user from critic detected.
Total prompt tokens: 151
Total completion tokens: 149
Duration: 2.55 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=61), content="Golden leaves in dance so fair,\nCrisp whispers brush the cooling air.\nPumpkin hues and sunset skies,\nAutumn's beauty fills our eyes.\n\nHarvest moons and firelight glow,\nWarmth amidst the evening's cold.\nNature's breath, a gentle sigh,\nFall's embrace bids summer goodbye.", type='TextMessage'), ToolCallRequestEvent(source='critic', models_usage=RequestUsage(prompt_tokens=123, completion_tokens=88), content=[FunctionCall(id='call_biBQto5IJvmwh1AR9utw0Dtc', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='critic', models_usage=None, content=[FunctionExecutionResult(content='Transfer to user.', call_id='call_biBQto5IJvmwh1AR9utw0Dtc')], type='ToolCallExecutionEvent'), HandoffMessage(sour